In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [11]:
from ml.models.base.brisk_xgboost import BriskXGBoost
from utils import helper, config, dasker
from dask_ml.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score


In [4]:
config.create_project_dirs(overwrite=True)

Cleared the existing saved models
Directory structure for project /covid created successfully


In [5]:
cl = dasker.get_local_client()
# cl =  dasker.get_global_client()
print(cl)

2022-10-25 19:04:58,893 - asyncio - DEBUG - Using selector: EpollSelector


2022-10-25 19:04:59,963 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-86kn9hdn', purging
2022-10-25 19:04:59,963 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-5_mc_w8o', purging
2022-10-25 19:04:59,964 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-paqjvqq7', purging
2022-10-25 19:04:59,964 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-r4miuuam', purging


<Client: 'tcp://127.0.0.1:8786' processes=4 threads=4, memory=15.37 GiB>


In [6]:
X, y = helper.get_covid_dataset()
X = X.drop(['location'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=config.rand_state)

In [25]:
brisk_xgboost = BriskXGBoost()
brisk_xgboost.X_train = X_train
brisk_xgboost.X_test = X_test
brisk_xgboost.y_train = y_train
brisk_xgboost.y_test = y_test

brisk_xgboost.n_trails = 500    #### increase this number to imporve the goodness of fit

In [26]:
best_xgboost = brisk_xgboost.fetch_model(retrain=True)

2022-10-25 19:07:56,133 - console_info - INFO - brisk xgboost: Starting train for trials:500 with boosted rounds: 250
2022-10-25 19:07:56,133 - console_info - INFO - brisk xgboost: Starting train for trials:500 with boosted rounds: 250
2022-10-25 19:07:56,141 - console_info - INFO - brisk xgboost: Cleared previous models in the model save path
2022-10-25 19:07:56,141 - console_info - INFO - brisk xgboost: Cleared previous models in the model save path
2022-10-25 19:12:40,564 - console_info - INFO - brisk xgboost: Number of trials: 500
2022-10-25 19:12:40,564 - console_info - INFO - brisk xgboost: Number of trials: 500
2022-10-25 19:12:40,567 - console_info - INFO - Best trial:
2022-10-25 19:12:40,567 - console_info - INFO - Best trial:
2022-10-25 19:12:40,620 - console_info - INFO -   xgboost Params: 
2022-10-25 19:12:40,620 - console_info - INFO -   xgboost Params: 
2022-10-25 19:12:40,625 - console_info - INFO -     booster gbtree
2022-10-25 19:12:40,625 - console_info - INFO -     b

In [27]:
brisk_xgboost.get_model_score(score_func=r2_score)

[0.9192925397105121, -0.4105869326458256]

In [28]:
pred = brisk_xgboost.predict(X_test)

In [29]:
r2_score(pred, y_test)
#mean_squared_error(pred, y)

-0.4105869326458256